In [23]:
import numpy as np
import pandas as pd
import os

import requests
from  bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By

from time import sleep
import random

In [24]:
# Vamos a entrar con selenium a la página oficial de la NBA.
# Si entramos desde España directamente a www.nba.com, nos redirecciona a una web oficial en español, por ello entramos a nba.com/games

chrome_driver = "chromedriver.exe"

browser = webdriver.Chrome()

browser.get("https://nba.com/games")

browser.maximize_window()

# Vamos a aceptar la política de cookies / privacidad con selenium.
# Vamos a ponerle que espere un segundo una vez que cargue todo correctamente antes de dar aceptar.

sleep(1)
browser.find_element(by=By.ID, value="onetrust-accept-btn-handler").click()

# Lo primero que queremos hacer es acceder a la página de estadísticas de los jugadores, por ello, a través de selenium vamos a ir a ese sitio específico

sleep(2)
browser.find_element(by=By.XPATH, value="//span[contains(text(), 'Players')]").click()

# Una vez estamos en la página de players, queremos que nos liste todos los jugadores que hay en la NBA esta temporada

seleccion_menu = browser.find_element(by=By.CSS_SELECTOR, value=".DropDown_select__4pIg9")
sleep(0.3)
seleccion_menu_all = browser.find_element(by=By.CSS_SELECTOR, value="#__next > div.Layout_base__6IeUC.Layout_justNav__2H4H0.Layout_withSubNav__ByKRF > div.Layout_mainContent__jXliI > main > div.MaxWidthContainer_mwc__ID5AG > section > div > div.PlayerList_content__kwT7z > div.PlayerList_filters__n_6IL > div.PlayerList_pagination__c5ijE > div > div.Pagination_pageDropdown__KgjBU > div > label > div > select > option:nth-child(1)").click()



In [25]:
# Ahora ya tenemos listado 542 filas. Vamos a empezar con el webscreaping
# Lo primero que tenemos que hacer es darle a bs4 la página que quiere hacer scraping, para ello vamos a preguntar a selenium cual es el html:

web_jugadores = browser.page_source

# A partir de ahí vamos con bs4

soup = BeautifulSoup(web_jugadores, "html.parser")

# Y ahora vamos a buscar todos los enlaces que hay en la página de los jugadores. Para ello utilizamos un bucle con la clase Anchor... para guardarlos en una variable

urls_jugadores_nba = [a["href"] for a in soup.find_all("a", class_="Anchor_anchor__cSc3P RosterRow_playerLink__qw1vG")]


In [26]:
print(urls_jugadores_nba)

['/player/1630173/precious-achiuwa/', '/player/203500/steven-adams/', '/player/1628389/bam-adebayo/', '/player/1630534/ochai-agbaji/', '/player/1630583/santi-aldama/', '/player/1629638/nickeil-alexander-walker/', '/player/1628960/grayson-allen/', '/player/1628386/jarrett-allen/', '/player/1630631/jose-alvarado/', '/player/203937/kyle-anderson/', '/player/203507/giannis-antetokounmpo/', '/player/203648/thanasis-antetokounmpo/', '/player/1630175/cole-anthony/', '/player/1628384/og-anunoby/', '/player/1627853/ryan-arcidiacono/', '/player/1630166/deni-avdija/', '/player/1629028/deandre-ayton/', '/player/1628962/udoka-azubuike/', '/player/1630641/ibou-badji/', '/player/1628963/marvin-bagley-iii/', '/player/1641735/amari-bailey/', '/player/1631116/patrick-baldwin-jr/', '/player/1630163/lamelo-ball/', '/player/1628366/lonzo-ball/', '/player/1628964/mo-bamba/', '/player/1631094/paolo-banchero/', '/player/1630217/desmond-bane/', '/player/1630625/dalano-banton/', '/player/1631230/dominick-barlow

In [27]:
# Al intentar acceder a los jugadores, nos hemos dado cuenta que nos daba un error por no tener la ruta completa. Así que vamos a añadirla antes de continuar:

url_base = "https://www.nba.com"
urls_jugadores_nba = [url_base + a["href"] for a in soup.find_all("a", class_="Anchor_anchor__cSc3P RosterRow_playerLink__qw1vG")]

# Solucionado este problema, ahora vamos a acceder a cada uno de los jugadores
response = requests.get(urls_jugadores_nba[0])

soup = BeautifulSoup(response.text, "html.parser")

In [28]:
def extraer_informacion_jugador(soup,url):
    id_jugador = url.split('/')[-3]
    elementos_nombre = soup.select(".PlayerSummary_playerNameText___MhqC")
    nombre = elementos_nombre[0].get_text() if len(elementos_nombre) >= 2 else "none"
    apellido = elementos_nombre[1].get_text() if len(elementos_nombre) >= 2 else "none"

    equipo_dorsal_posicion = soup.find("p", class_="PlayerSummary_mainInnerInfo__jv3LO")
    if equipo_dorsal_posicion:
        equipo_dorsal_posicion = equipo_dorsal_posicion.get_text()
        partes = equipo_dorsal_posicion.split('|')
        equipo = partes[0].strip() if len(partes) > 0 else "none"
        dorsal = partes[1].strip() if len(partes) > 1 else "none"
        posicion = partes[2].strip() if len(partes) > 2 else "none"
    else:
        equipo = dorsal = posicion = ""
   
    
    datos_varios = soup.find_all("p", class_="PlayerSummary_playerStatValue___EDg_")
    ppg = datos_varios[0].get_text() if len(datos_varios) > 0 else "none"
    rpg = datos_varios[1].get_text() if len(datos_varios) > 1 else "none"
    apg = datos_varios[2].get_text() if len(datos_varios) > 2 else "none"
    pie = datos_varios[3].get_text() if len(datos_varios) > 3 else "none"

    detalles = [p.get_text() for p in soup.find_all("p", class_="PlayerSummary_playerInfoValue__JS8_v")]
    altura = detalles[0] if len(detalles) > 0 else "none"
    peso = detalles[1] if len(detalles) > 1 else "none"
    pais = detalles[2] if len(detalles) > 2 else "none"
    ultimo_equipo = detalles[3] if len(detalles) > 3 else "none"
    edad = detalles[4] if len(detalles) > 4 else "none"
    cumpleaños = detalles[5] if len(detalles) > 5 else "none"
    draft = detalles[6] if len(detalles) > 6 else "none"
    experiencia = detalles[7] if len(detalles) > 7 else "none"

    return {
        "id_jugador": id_jugador,
        "nombre": nombre,
        "apellido": apellido,
        "equipo": equipo,
        "dorsal": dorsal,
        "posicion": posicion,
        "ppg": ppg,
        "rpg": rpg,
        "apg": apg,
        "pie": pie,
        "altura": altura,
        "peso": peso,
        "pais": pais,
        "ultimo_equipo": ultimo_equipo,
        "edad": edad,
        "cumpleaños": cumpleaños,
        "draft": draft,
        "experiencia": experiencia
    }

In [29]:
id_jugador, nombre, apellido, equipo, dorsal, posicion, ppg, rpg, apg, pie, altura, peso, pais, ultimo_equipo, edad, cumpleaños, draft, experiencia = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []


In [34]:
for url in urls_jugadores_nba:
    print(url)
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Para extraer la información de los jugadores vamos a utilizar la función que definimos la de "extraer_informacion_jugador"
    info_jugador = extraer_informacion_jugador(soup,url)

    # Y ahora vamos a agregar toda esa información utilizando un append a la lista
    id_jugador.append(info_jugador['id_jugador'])
    nombre.append(info_jugador['nombre'])
    apellido.append(info_jugador['apellido'])
    equipo.append(info_jugador['equipo'])
    dorsal.append(info_jugador['dorsal'])
    posicion.append(info_jugador['posicion'])
    ppg.append(info_jugador['ppg'])
    rpg.append(info_jugador['rpg'])
    apg.append(info_jugador['apg'])
    pie.append(info_jugador['pie'])
    altura.append(info_jugador['altura'])
    peso.append(info_jugador['peso'])
    pais.append(info_jugador['pais'])
    ultimo_equipo.append(info_jugador['ultimo_equipo'])
    edad.append(info_jugador['edad'])
    cumpleaños.append(info_jugador['cumpleaños'])
    draft.append(info_jugador['draft'])
    experiencia.append(info_jugador['experiencia'])
    
    # Para intentar que el servidor no nos bloquee, vamos a introducir un rango aleatorio entre
    # 3 y 10 segundos para hacer peticiones, igual es mucho, pero con menos nos ha bloqueado la web de la NBA
    tiempo_espera = random.uniform(3, 10)
    sleep(tiempo_espera)


https://www.nba.com/player/1630173/precious-achiuwa/
https://www.nba.com/player/203500/steven-adams/
https://www.nba.com/player/1628389/bam-adebayo/
https://www.nba.com/player/1630534/ochai-agbaji/
https://www.nba.com/player/1630583/santi-aldama/
https://www.nba.com/player/1629638/nickeil-alexander-walker/
https://www.nba.com/player/1628960/grayson-allen/
https://www.nba.com/player/1628386/jarrett-allen/
https://www.nba.com/player/1630631/jose-alvarado/
https://www.nba.com/player/203937/kyle-anderson/
https://www.nba.com/player/203507/giannis-antetokounmpo/
https://www.nba.com/player/203648/thanasis-antetokounmpo/
https://www.nba.com/player/1630175/cole-anthony/
https://www.nba.com/player/1628384/og-anunoby/
https://www.nba.com/player/1627853/ryan-arcidiacono/
https://www.nba.com/player/1630166/deni-avdija/
https://www.nba.com/player/1629028/deandre-ayton/
https://www.nba.com/player/1628962/udoka-azubuike/
https://www.nba.com/player/1630641/ibou-badji/
https://www.nba.com/player/162896

In [35]:
df_jugadores = pd.DataFrame({
    "id_jugador": id_jugador,
    "Nombre": nombre,
    "Apellido": apellido,
    "Equipo": equipo,
    "Dorsal": dorsal,
    "Posición": posicion,
    "PPG": ppg,
    "RPG": rpg,
    "APG": apg,
    "PIE": pie,
    "Altura": altura,
    "Peso": peso,
    "País": pais,
    "Último Equipo": ultimo_equipo,
    "Edad": edad,
    "Cumpleaños": cumpleaños,
    "Draft": draft,
    "Experiencia": experiencia
})

In [36]:
# Me he dado cuenta que igual hacemos el proceso 3-4 veces para ir actualizando datos o por si hay algún
# dato que a través de bs4 no se hay aimportado bien. Entonces se me ha ocurrido que no sobreescriba el archivo si
# ya existe, y así luego con pandas, podemos hacer actualización de archivos o alguna cosa de esas
# Para ello se puede hacer lo del contador que ya he utilizado en otro sitio para generar nombres de excel


def guardar_excel_con_numeracion(df, directorio, nombre_base):
    directorio = os.path.join(os.getcwd(), "excels")

    # Si no existe el directorio con esto lo creará
    if not os.path.exists(directorio):
        os.makedirs(directorio)

    ruta_base = os.path.join(directorio, nombre_base)
    contador = 0
    ruta_final = f"{ruta_base}.xlsx"

    while os.path.exists(ruta_final):
        contador += 1
        ruta_final = f"{ruta_base}_{contador}.xlsx"

    df.to_excel(ruta_final, index=False, engine='openpyxl')
    print(f"Archivo guardado como: {ruta_final}")

guardar_excel_con_numeracion(df_jugadores, "excels", "jugadores_nba")


Archivo guardado como: c:\Users\34670\Desktop\python\Hack a boss\proyecto_1\excels\jugadores_nba_4.xlsx


In [172]:
def llamando_web_nba_jugadores():
    # Iniciar el navegador
    browser = webdriver.Chrome()

    # Navegar a la página
    browser.get("https://nba.com/games")
    browser.maximize_window()

    # Aceptar la política de cookies
    sleep(1)
    browser.find_element(by=By.ID, value="onetrust-accept-btn-handler").click()

    # Navegar a la página de jugadores
    sleep(2)
    browser.find_element(by=By.XPATH, value="//span[contains(text(), 'Players')]").click()

    # Seleccionar para mostrar todos los jugadores
    sleep(0.3)
    seleccion_menu = browser.find_element(by=By.CSS_SELECTOR, value=".DropDown_select__4pIg9")
    seleccion_menu.click()
    seleccion_menu_all = browser.find_element(by=By.CSS_SELECTOR, value="#__next > div.Layout_base__6IeUC.Layout_justNav__2H4H0.Layout_withSubNav__ByKRF > div.Layout_mainContent__jXliI > main > div.MaxWidthContainer_mwc__ID5AG > section > div > div.PlayerList_content__kwT7z > div.PlayerList_filters__n_6IL > div.PlayerList_pagination__c5ijE > div > div.Pagination_pageDropdown__KgjBU > div > label > div > select > option:nth-child(1)").click()

    # Esperar a que la lista de jugadores se cargue
    sleep(2)

       # Ahora ya tenemos listado 542 filas. Vamos a empezar con el webscreaping
    # Lo primero que tenemos que hacer es darle a bs4 la página que quiere hacer scraping, para ello vamos a preguntar a selenium cual es el html:

    web_jugadores = browser.page_source

    # A partir de ahí vamos con bs4

    soup = BeautifulSoup(web_jugadores, "html.parser")

    # Y ahora vamos a buscar todos los enlaces que hay en la página de los jugadores. Para ello utilizamos un bucle con la clase Anchor... para guardarlos en una variable

    urls_jugadores_nba = [a["href"] for a in soup.find_all("a", class_="Anchor_anchor__cSc3P RosterRow_playerLink__qw1vG")]
    
    return web_jugadores

# Ahora puedes llamar a la función
web_jugadores = llamando_web_nba_jugadores()